<a href="https://colab.research.google.com/github/FLjv77/ML_Physics/blob/main/MachineLearningInPhysics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Algorithm for Clustering

Input Params:


*   Dataset
*   Metrics
*   Score function (seems be fixed)
*   Number of clusters range

Output:


*   Best clusters and it's score




In [ ]:
# for each possible number of cluster as c_n
# select c_n point randomly
# find nearest point according metric in each cluster
# update cluster center by metric avarage of the nodes in same cluster
# do this untill all nodes covered
# calculate score of clustering
#
#
# FOR EXAMPLE
# metric         => aghlidosian distance, relative entropy of coherence, tangle
# score function => distance of cluster from each other / density data in each cluster
#
#
#
#
#
#
